# Scikit-Learn IRIS Model

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

## Train locally
 

In [1]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()


Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in IrisClassifier.sav
Model saved!


/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Wrap model using s2i

## REST test

In [2]:
!s2i build . seldonio/seldon-core-s2i-python37:0.18 sklearn-iris:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [3]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 sklearn-iris:0.1

183e42c61200a89eae242a9400ff03f0c74a9405c2d022e7a01ecb3542d36380


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[5.964 4.006 2.081 1.031]]
RECEIVED RESPONSE:
meta {
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.9548873249364169
        }
        values {
          number_value: 0.04505474761561406
        }
        values {
          number_value: 5.7927447968952436e-05
        }
      }
    }
  }
}




In [5]:
!docker rm iris_predictor --force

iris_predictor


## grpc test

In [6]:
!s2i build -E .s2i/environment_grpc . seldonio/seldon-core-s2i-python3:0.18 sklearn-iris:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [7]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 sklearn-iris:0.1

d84a589e03dd9c5461d913f093834b511e1593749cf8be9794985a582b8094a0


Test using NDArray payload

In [8]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:

[[6.002 3.459 7.38  0.827]]
RECEIVED RESPONSE:
meta {
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  ndarray {
    values {
      list_value {
        values {
          number_value: 3.41671271543092e-05
        }
        values {
          number_value: 0.43959527937639553
        }
        values {
          number_value: 0.5603705534964502
        }
      }
    }
  }
}




Test using Tensor payload

In [9]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc --tensor

----------------------------------------
SENDING NEW REQUEST:

[[7.076 2.11  5.33  2.677]]
RECEIVED RESPONSE:
meta {
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  tensor {
    shape: 1
    shape: 3
    values: 0.0001347135284905143
    values: 0.34256764312153126
    values: 0.6572976433499783
  }
}




In [10]:
!docker rm iris_predictor --force

iris_predictor


## Test using Minikube

Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)

In [12]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

## Wrap Model and Test

In [11]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.18 sklearn-iris:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [12]:
!kubectl create -f sklearn_iris_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [13]:
!kubectl rollout status deploy/sklearn-iris-deployment-sklearn-iris-predictor-a5a7453

Waiting for deployment "sklearn-iris-deployment-sklearn-iris-predictor-a5a7453" rollout to finish: 0 of 1 updated replicas are available...
deployment "sklearn-iris-deployment-sklearn-iris-predictor-a5a7453" successfully rolled out


In [14]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    seldon-deployment-example --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[6.956 4.036 9.426 2.103]]
RECEIVED RESPONSE:
meta {
  puid: "if585togj6ra3nb4tt5c1vrccg"
  requestPath {
    key: "sklearn-iris-classifier"
    value: "sklearn-iris:0.1"
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  ndarray {
    values {
      list_value {
        values {
          number_value: 3.6116624137419636e-07
        }
        values {
          number_value: 0.3368495629835383
        }
        values {
          number_value: 0.6631500758502203
        }
      }
    }
  }
}




In [25]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
